<a href="https://colab.research.google.com/github/ARAN1218/piedpiper-python/blob/main/PPP%E2%91%A5_Share_price_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 株価予測でお金持ちになろう（！？）

In [ ]:
!pip install yfinance
!pip install mplfinance
!pip install plotly

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import plotly.graph_objects as go
import plotly.figure_factory as ff

# 株価分析に役立つライブラリ
import yfinance as yf
import mplfinance as mpf

In [ ]:
# 任天堂の株価情報をAPIを用いて簡単に取得できる！
# みんなもティアーズオブキングダム買おう！！
ticker = "7974.T"
df_all = yf.download(ticker, start= "2011-01-01",end="2021-12-31", interval = "1d")
df_all.head()

カラムの意味
- Date：取得日時
- Open：始値
- High：高値
- Low：底値
- Close：終値
- Adj Close：調整後終値...株式分割の実施の前後で株価を連続的にとらえるために、分割実施前の終値を分割後の値に調整した終値
- Volume：出来高...株式の取引量

In [ ]:
# pandasのプロット
df_all.plot()

In [ ]:
# mplfinanceによるプロット
mpf.plot(df_all, type="candle",volume=True, figratio=(10,5))

# 特徴量エンジニアリング・データ分析

In [ ]:
# ほぼ答えを表しているAdj Closeは消す
del df_all['Adj Close']

In [ ]:
# 今回の目的変数：1日後の調整済み終値を作る
df_all['Close_next'] = df_all['Close'].shift(-1)
df_all.tail()

In [ ]:
# 前日との差
df_all['Close_diff_pre1'] = df_all['Close'].diff(1)
df_all.head()

In [ ]:
# 移動平均
SMA1 = 5   #短期5日
SMA2 = 10  #中期10日
SMA3 = 15  #長期15日
df_all['SMA5'] = df_all['Close'].rolling(SMA1).mean() #短期移動平均の算出
df_all['SMA10'] = df_all['Close'].rolling(SMA2).mean() #中期移動平均の算出
df_all['SMA15'] = df_all['Close'].rolling(SMA3).mean() #長期移動平均の算出
df_all.head(20)

In [ ]:
# 欠損値を消す
df_all = df_all.dropna(how='any')
df_all.head()

In [ ]:
import plotly.graph_objects as go
df_corr = df_all.corr() #相関行列を生成

fig = go.Figure()
fig.add_trace(
    go.Heatmap(
        x = df_corr.columns,
        y = df_corr.index,
        z = np.array(df_corr) 
    )
)

これでも視覚的に分かりやすいのですが、テキストとして相関係数が書かれていたり、色のグラデーションでより直感的に相関の大小関係が分かった方が良いですよね？  
以下でこのヒートマップを改善してみましょ〜

In [ ]:
# より見やすく！！！
import plotly.figure_factory as ff
x = list(df_corr.columns)
y = list(df_corr.index)
z = np.array(df_corr)

fig = ff.create_annotated_heatmap(
    z,
    x = x,
    y = y ,
    annotation_text = np.around(z, decimals=3),
    hoverinfo='z',
    colorscale='OrRd'
    )
fig.show()

最初のやつと比べて、相関係数を表示＆カラースケールを白→赤の順に変化させるようにしました！  
本当はもっと良い感じに見やすくなるのですが、今回は相関が高すぎて真っ赤ですね...

# モデリング
まずは重回帰分析から

In [ ]:
from sklearn.linear_model import LinearRegression
# 時系列分割のためTimeSeriesSplitのインポート
from sklearn.model_selection import TimeSeriesSplit
# 予測精度検証のためMSEをインポート
from sklearn.metrics import mean_squared_error as mse

In [ ]:
train = df_all[:'2021-01-01']
test = df_all['2021-01-01':]

# 学習用データとテストデータそれぞれを説明変数と目的変数に分離する
X_train = train.drop('Close_next', axis=1) #学習用データ説明変数
y_train = train['Close_next'] #学習用データ目的変数
X_test = test.drop('Close_next', axis=1) #テストデータ説明変数
y_test = test['Close_next'] #テストデータ目的変数

In [ ]:
# 時系列分割交差検証
valid_scores = []
tscv = TimeSeriesSplit(n_splits=4)
for fold, (train_indices, valid_indices) in enumerate(tscv.split(X_train)):
    X_train_cv, X_valid_cv = X_train.iloc[train_indices], X_train.iloc[valid_indices]
    y_train_cv, y_valid_cv = y_train.iloc[train_indices], y_train.iloc[valid_indices]
    # 線形回帰モデルのインスタンス化
    model = LinearRegression()
    # モデル学習
    model.fit(X_train_cv, y_train_cv)
    # 予測
    y_valid_pred = model.predict(X_valid_cv)
    # 予測精度(RMSE)の算出
    score = np.sqrt(mse(y_valid_cv, y_valid_pred))
    # 予測精度スコアをリストに格納
    valid_scores.append(score)

In [ ]:
print(f'valid_scores: {valid_scores}')
cv_score = np.mean(valid_scores)
print(f'CV score: {cv_score}')

In [ ]:
# 可視化
model = LinearRegression()
model.fit(X_train, y_train)
df_result = test[['Close_next']]
df_result['Close_pred'] = model.predict(X_test)
df_result

In [ ]:
print('RMSE：', np.sqrt(mse(df_result['Close_next'], df_result['Close_pred'])))
df_result.plot()

これで、みんなもお金持ちですね！（）

# ~このAIが使えない理由~

1. 株価は人の心理が動かしている面が強く、不足の事態として急激な値動きに対応できていない。(拡大すると分かりやすい)
2. 考慮している変数が少なすぎて、株価を左右する要因をカバーしきれていない。（例えば、その企業の取引先の企業の株価だったり、日経平均等を用いた日本全体の景気等）

In [ ]:
# 拡大してみてみると...？
df_result.iloc[:20,:].plot()
# 全然ダメで草

上の予測結果を完全に信じて挑戦すると、1/6とか1/10辺りでドツボにハマりそうですね（笑）

そもそも今回使った重回帰分析は一体何をしているのかというと、以下のような多項式を全データに対して最も誤差が小さくなる（**最小二乗法**）ように求めると言うことをやっています。

$$y = \beta + \alpha_1 x_1 + \alpha_2 x_2 + ... + \alpha_n x_n$$

$$y...目的変数（次の日の終値）$$
$$\beta ... 切片$$
$$\alpha ... 偏回帰係数$$
$$x...説明変数（前の日の終値等）$$

数式アレルギーの人がいるかもしれないので言葉で説明すると、「モデルに入力した色々な値に係数を掛けて翌日の株価を再現しやすい式を作る」ということを説明しています。  
今回は前の日の終値が次の日の終値に割と近い値を取りますよね？つまり、前の日の終値に1に近い係数を掛けておけば、概ね「精度良く」予測できる重回帰モデルが作れてしまうのです。  
しかし、そんなもの私たちは望んでないですよね？私たちが欲しいのは総合的に考えて明日の終値がどのような値になるのかを出力してくれるやつのはずです。

でも残念ながら、重回帰分析の仕組み上そんなことはできないのです...

In [ ]:
# 試しに今回のモデルの偏回帰係数・切片を見てみると...？

model_params = list(model.coef_).copy() # 偏回帰係数
model_params.append(model.intercept_) # 切片
columns = list(X_train.columns).copy()
columns.append("intercept")
df_params = pd.DataFrame(model_params, index=columns).T

df_params.plot.bar()

ほら見たことか。やっぱりしょーもないモデルでしたね🐦

じゃあもっとマシなアプローチはあるの？というと、上下どちらに動くかというやや単純化したタスクの予測させるAIを強化学習というアプローチで作成して買い付けを行わせたり、時系列回帰が得意なモデルにデータを食わせたりすることがあるそうです。  
**（私は責任を負いませんが）**皆さんも頑張って株価を予測させて、是非お金持ちになってください🐦

# オマケ：時系列モデル「ARIMA」で予測

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
# ARIMAモデル実装
model = ARIMA(y_train, order=(6,1,0))
model_fit = model.fit()
print(model_fit.summary())

In [ ]:
history = [x for x in y_train]
model_predictions = []
for time_point in range(len(y_test)):
    #ARIMAモデル 実装
    model = ARIMA(history, order=(6,1,0))
    model_fit = model.fit()
    #予測データの出力
    output = model_fit.forecast()
    yhat = output[0]
    model_predictions.append(yhat)
    #トレーニングデータの取り込み
    true_test_value = y_test[time_point]
    history.append(true_test_value)

In [ ]:
# pred = pd.DataFrame(model_predictions, columns=['Close'])
# pred = pred.set_index(df_all.index)
df_result = test[['Close_next']]
df_result['Close_pred'] = np.array(model_predictions)
df_result

In [ ]:
print('RMSE：', np.sqrt(mse(df_result['Close_next'], df_result['Close_pred'])))
df_result.plot()
# ちょっとだけ良くなった？

# 参考文献
- Pythonで予測｜株価はいくらになる？【機械学習を使って予測する方法を解説：データ取得、データ前処理、モデル作成、モデル評価まで】、キノコード、https://kino-code.com/python-stock-price2/
- 【実践】Pythonで株の取引データを分析し予測してみよう！、スタビジ、https://toukei-lab.com/python_stock

# LICENCE

このcolabは[githubのREADME](https://github.com/agu-piedpiper/piedpiper-python)にて記載の通りのライセンスに従います。